In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
import os
path = "/content/gdrive/MyDrive/JPM/UK_data/UK_Speech/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [15]:
speech_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('â','')
  text=text.replace('ï','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/Pages/speeches/default.aspx','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/speeches','')
  if len(text.split(' '))>100:
    speech_text[file_list[i][:8]]='speech '+text
  else:
    print(i,len(text.split(' ')),text)
  #print(text)

36 84 Page 1 Inclusive Capitalism Conference: In Conversation with Governor Mark Carney Given at Mansion House, LondonPage 2 Published on 26 June 2015 Governor Mark Carney participates in an on-stage conversation with The Economists Editor-in- Chief Zanny Minton-Beddoes at the Conference on Inclusive Capitalism: The Pathway to Action. The Governor discusses how we can create core financial markets that work fairly and effectively to deliver strong, sustainable and balanced growth. Mark Carney Former Governor, Bank of England from 2013 to 2020 Â©2022 Bank of England
308 32 Page 1 Consumption and Interest Rates - speech by Tim Besley Given at Centre for Economic Policy ResearchPage 2 Published on 19 July 2007 Consumption and Interest Rates Â©2022 Bank of England
455 29 Page 1 Interpreting Monetary Policy - speech by David Miles Speech given at Imperial College, LondonPage 2 Published on 25 February 2010 Interpreting Monetary Policy Â©2022 Bank of England
633 53 Page 1 Momentum in the ho

In [16]:
import os
path = "/content/gdrive/MyDrive/JPM/UK_data/UK_minutes/"
os.chdir(path)
os.listdir(path)
file_list=[i for i in os.listdir(path) if 'txt' in i]

In [17]:
minutes_text=dict()
for i in range(len(file_list)):
  f=open(path+file_list[i],'r',encoding='latin1')
  f1=f.readlines()
  contents=[]
  for line in f1:
    if ('[SECTION]' not in line):
      if len(line.strip().split(' '))>=5:
        if ('Voting for the FOMC monetary policy action were' in line):
          break
        contents.append(line.strip())
  text = ' '.join(contents)
  text=text.replace('â','')
  text=text.replace('ï','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/Pages/speeches/default.aspx','')
  text=text.replace('All speeches are available online at www.bankofengland.co.uk/publications/speeches','')
  if len(text.split(' '))>100:
    minutes_text[file_list[i][:8]]='minutes '+text
  else:
    print(i,len(text.split(' ')),text)
  #print(text)

8 10 Monetary policy summary and minutes Monetary Policy Committee voting history
12 1 
50 1 
55 21 The Governor invited the Committee to vote on the propositions that: The Committee voted unanimously in favour of all four propositions.


In [18]:
raw_data_speech=pd.DataFrame({'date':speech_text.keys(),'text':speech_text.values()})
raw_data_speech.sort_values(by='date',inplace=True)
raw_data_speech

,date,text
181,19970225,speech Speech by the Governor at the Queen Eli...
157,19970612,speech Lord Mayors Banquet Speech given by...
371,19970624,speech The Sixteenth Mais Lecture Speech gi...
990,19971029,speech The Inflation Target five years on L...
128,19971118,speech Developments in UK financial markets ...
...,...,...
396,20220223,speech The economy and policy trade-offs Sp...
572,20220301,speech The effectiveness of vaccination again...
26,20220330,speech Bank of England Page 1 Technica...
573,20220331,speech Page 1 The road to enhanced payments - ...


In [19]:
raw_data_statement=pd.DataFrame({'date':minutes_text.keys(),'text':minutes_text.values()})
raw_data_statement.sort_values(by='date',inplace=True)
raw_data_statement

,date,text
62,19970716,minutes I Monetary conditions 2 The 12-mon...
64,19970813,minutes I Summary of the analysis presented to...
63,19971015,minutes 1 The meeting took place against the ...
1,19971112,minutes Publication date: 12 November 1997 8 ...
0,19971210,minutes Publication date: 10 December 1997 5 ...
...,...,...
59,20210923,minutes The Bank of EnglandÂs Monetary Policy...
45,20211104,minutes The Bank of EnglandÂs Monetary Policy...
38,20211216,minutes The Bank of EnglandÂs Monetary Policy...
61,20220203,minutes The Bank of EnglandÂs Monetary Policy...


In [63]:
raw_data=raw_data_speech#pd.concat([raw_data_statement,raw_data_speech],axis=0)
raw_data.sort_values(by='date',inplace=True)

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
eng_stopwords = stopwords.words('english')

# 去掉html标签
# 去掉非英文字符
# 去停用词
# 重新组合为句子
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)

raw_data['clean_review'] = raw_data.text.apply(clean_text)
raw_data.head()

,date,text,clean_review,change
181,19970225,speech Speech by the Governor at the Queen Eli...,speech speech governor queen elizabeth confere...,0.006020
157,19970612,speech Lord Mayors Banquet Speech given by...,speech lord mayor banquet speech given edward ...,0.015788
371,19970624,speech The Sixteenth Mais Lecture Speech gi...,speech sixteenth mais lecture speech given edw...,-0.014771
990,19971029,speech The Inflation Target five years on L...,speech inflation target five years lecture del...,-0.000132
128,19971118,speech Developments in UK financial markets ...,speech developments uk financial markets speec...,-0.018535


In [66]:
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_30_UK.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
interest_rate.index=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+10,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data[raw_data.date>'19980101'].dropna()

In [67]:
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))


In [68]:
interest_rate.index=list(range(len(interest_rate)))

In [69]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
def setup_seed(seed=0):
    import torch
    import os
    import numpy as np
    import random
    torch.manual_seed(seed)  # 为CPU设置随机种子
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    if torch.cuda.is_available():
        # torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        torch.cuda.manual_seed(seed)  # 为当前GPU设置随机种子
        torch.cuda.manual_seed_all(seed)  # 为所有GPU设置随机种子
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import nltk
import re
from nltk.corpus import stopwords
import numpy as np
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')
def clean_text(text):
    #text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    words = [w for w in words if w not in eng_stopwords]
    return ' '.join(words)
def sliding_windows(data,prediction, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = prediction[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
from sklearn.metrics import mean_squared_error


def taining(train_X,train_Y,test_X,test_Y,data,ratio,seq_length,features,l2,seed):
  #train_X,train_Y,test_X,test_Y=split_data(data,ratio,features,seq_length)
  num_epochs = 400
  learning_rate = 0.01
  input_size = features
  hidden_size = 2
  num_layers = 1
  num_classes = 1
  setup_seed(seed)
  lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
  criterion = torch.nn.MSELoss()    # mean-squared error for regression
  optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate,weight_decay=l2)
  from sklearn.metrics import mean_squared_error
  for epoch in range(num_epochs):
      outputs = lstm(train_X)
      optimizer.zero_grad()
      loss = criterion(outputs, train_Y[0])
      loss.backward()
      optimizer.step()
      predTrain = lstm(test_X)
      print(predTrain.shape)
      tmp=pd.DataFrame(list(test_Y))
      tmp["1"]=list(predTrain.detach().numpy().reshape(1,-1))[0]
      r = mean_squared_error(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])
      print('IC',round(tmp.corr().iloc[0,1],5),
            'IC spear',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[0],5),\
            'P_value',round(stats.spearmanr(test_Y,list(predTrain.detach().numpy().reshape(1,-1))[0])[1],5),
            'loss',round(r,5))
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [74]:
features=200
seq_length = 1
ratio_=0.75
periods=5
interest_rate=pd.read_csv('/content/gdrive/MyDrive/JPM/Interest Rate/Interest_10_UK.csv',index_col=[0])
interest_rate=interest_rate.sort_values(by='Date')
interest_rate['range']=list(range(len(interest_rate)))
interest_rate.index=list(range(len(interest_rate)))
raw_data['change']=np.nan
for i in range(len(raw_data)):
  try:
    index=int(interest_rate[interest_rate['Date']<=int(raw_data.iloc[i,0])]['range'].tail(1))
    raw_data.iloc[i,-1]=(interest_rate.iloc[index+periods,1]-interest_rate.iloc[index,1])/(1+interest_rate.iloc[index,1])
  except:
    pass
new_raw=raw_data[raw_data.date>'19980101'].dropna()
vectorizer_freq = CountVectorizer(max_features = features) 
total_vsm_freq = vectorizer_freq.fit_transform(new_raw.clean_review).toarray()
print("以词频为元素的文本-单词矩阵的维度是：\n\n",total_vsm_freq.shape)
from scipy import stats
vectorizer_tfidf=TfidfVectorizer(max_features=features)
total_vsm_tfidf=vectorizer_tfidf.fit_transform(new_raw.clean_review).toarray()
x, y = sliding_windows(total_vsm_tfidf,list(new_raw.change), seq_length)
train_size = int(len(x) * ratio_)
test_size = len(x) - train_size
train_Y=np.array(x[0:train_size])
from sklearn.preprocessing import RobustScaler
train_Y=np.array(y[0:train_size])
scaler =RobustScaler().fit(train_Y.reshape(-1, 1) )
train_Y = scaler.transform(train_Y.reshape(-1, 1)).reshape(1,-1)[0]
train_Y=Variable(torch.Tensor(train_Y))
train_X = Variable(torch.Tensor(np.array(x[0:train_size])))
test_X = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
test_Y = np.array(y[train_size:len(y)])

以词频为元素的文本-单词矩阵的维度是：

 (1050, 200)


In [75]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.75,5,200,0.001,5)

torch.Size([262, 1])
IC -0.0809 IC spear -0.05243 P_value 0.39805 loss 0.15167
Epoch: 0, loss: 0.33454
torch.Size([262, 1])
IC -0.07053 IC spear -0.03823 P_value 0.5378 loss 0.12945
Epoch: 1, loss: 0.30335
torch.Size([262, 1])
IC -0.0601 IC spear -0.0263 P_value 0.6718 loss 0.10837
Epoch: 2, loss: 0.27244
torch.Size([262, 1])
IC -0.0497 IC spear -0.0135 P_value 0.82779 loss 0.0887
Epoch: 3, loss: 0.24213
torch.Size([262, 1])
IC -0.03947 IC spear -0.00503 P_value 0.93537 loss 0.0707
Epoch: 4, loss: 0.21273
torch.Size([262, 1])
IC -0.02953 IC spear 0.00546 P_value 0.92998 loss 0.05457
Epoch: 5, loss: 0.18452
torch.Size([262, 1])
IC -0.02 IC spear 0.01485 P_value 0.8109 loss 0.0405
Epoch: 6, loss: 0.15779
torch.Size([262, 1])
IC -0.01097 IC spear 0.02368 P_value 0.70279 loss 0.02862
Epoch: 7, loss: 0.13277
torch.Size([262, 1])
IC -0.00253 IC spear 0.032 P_value 0.60607 loss 0.01903
Epoch: 8, loss: 0.10967
torch.Size([262, 1])
IC 0.00529 IC spear 0.0382 P_value 0.53812 loss 0.01179
Epoch: 

In [77]:
taining(train_X,train_Y,test_X,test_Y,new_raw,0.75,5,200,0.001,6)

torch.Size([262, 1])
IC 0.00928 IC spear 0.00725 P_value 0.90702 loss 0.08928
Epoch: 0, loss: 0.21417
torch.Size([262, 1])
IC 0.01552 IC spear 0.00683 P_value 0.91239 loss 0.08272
Epoch: 1, loss: 0.20408
torch.Size([262, 1])
IC 0.02072 IC spear 0.01001 P_value 0.8719 loss 0.07616
Epoch: 2, loss: 0.19381
torch.Size([262, 1])
IC 0.02551 IC spear 0.0128 P_value 0.83663 loss 0.06961
Epoch: 3, loss: 0.18335
torch.Size([262, 1])
IC 0.02998 IC spear 0.01711 P_value 0.78278 loss 0.06308
Epoch: 4, loss: 0.17269
torch.Size([262, 1])
IC 0.03409 IC spear 0.02252 P_value 0.71679 loss 0.05659
Epoch: 5, loss: 0.16183
torch.Size([262, 1])
IC 0.03779 IC spear 0.02479 P_value 0.68958 loss 0.05018
Epoch: 6, loss: 0.15078
torch.Size([262, 1])
IC 0.04103 IC spear 0.02476 P_value 0.68994 loss 0.04389
Epoch: 7, loss: 0.13959
torch.Size([262, 1])
IC 0.04381 IC spear 0.02666 P_value 0.66751 loss 0.03777
Epoch: 8, loss: 0.12830
torch.Size([262, 1])
IC 0.04611 IC spear 0.02829 P_value 0.64854 loss 0.03189
Epoch: